In [2]:
import os
import time
from datetime import timezone
from datetime import datetime, timedelta
import requests
import json
# from openai import OpenAI
from tenacity import (retry,stop_after_attempt,wait_random_exponential)
from bs4 import BeautifulSoup
import tiktoken
from email.message import EmailMessage
import ssl
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import io

import pretty_errors
import psycopg2
##################################################################
#env variables
from dotenv import load_dotenv
env_path = 'C:\\Users\\user\\Desktop\\My_Files\\Anaconda  Projects\\DevTechPro\\.env'
load_dotenv(dotenv_path=env_path)

OPENAI_API_KEY= os.getenv('OPENAI_API_KEY')
# os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
OLLAMA_API_KEY=os.getenv('OLLAMA_API_KEY')
MISTRALAI_API_KEY = os.getenv('MISTRALAI_API_KEY')
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')

POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
CLEARBIT_KEY = os.getenv('CLEARBIT_KEY')
HUBSPOT_ACCESS_TOKEN =  os.environ.get('HUBSPOT_ACCESS_TOKEN')
CLEARBIT_KEY = os.getenv('CLEARBIT_KEY')
AGENT_EMAIL = os.getenv('AGENT_EMAIL')
AGENT_EMAIL_PASSWORD= os.getenv('AGENT_EMAIL_PASSWORD')
SALES_EMAIL = 'rachad@bassemfawaz.de'

In [3]:
free_email_providers = ['gmail.com', 'yahoo.com', 'outlook.live.com', 'outlook.com', 'mail.aol.com', 'protonmail.com', ' zoho.com/mail/', 'gmx.com',' mail.com',' icloud.com',' tutanota.com', ] 
##################################################################

form_ids_names = {'b9368dcb-1c5a-40ac-8f1d-ee9589cbd103': 'Light Form',
                  '497a7397-0a4c-4702-8355-9eac16d41031': 'Inf  Test Form'}

openai = 'OpenAI'
azureopenai = 'AzureOpenAI'
mistralai = 'MistralAI'
ollama = 'Ollama'

# LLM_PROVIDER = 'MistralAI'
LLM_PROVIDER = 'OpenAI'
# LLM_PROVIDER = 'Ollama'
# LLM_PROVIDER = 'AzureOpenAI'

# MODEL_NAME = 'mistral-medium-latest'
# MODEL_NAME = 'codellama:13b'
MODEL_NAME = 'gpt-3.5-turbo-16k'
# MODEL_NAME = 'AzureOpenAI'

MODEL_MAX_TOKENS = 16000
tiktoken_MODEL_NAME = 'gpt-3.5-turbo-16k' #no need to change it is used only for counting nb of tokens

MODEL_NAME_scoring = 'gpt-3.5-turbo-0125'  # model used for scoring
MODEL_NAME_response = 'gpt-3.5-turbo-0125' # model used for creating email responses
MODEL_NAME_HTML = 'gpt-3.5-turbo-0125'     # model used for re writing emails in HTML

In [4]:
import requests
import time
import IPython.display as ipd 

In [29]:
def call_openai_tts(text, model='tts-1', voice='alloy', max_retries=6, backoff_factor=1.5, openai_api_key=OPENAI_API_KEY, print_results=False):
    if not openai_api_key:
        raise ValueError("OpenAI API key is required.")

    for attempt in range(max_retries):
        try:
            url = 'https://api.openai.com/v1/audio/speech'
            payload = {
                'model': model,
                'voice': voice,
                'input': text
            }
            headers = {
                'Content-Type': 'application/json',
                'Authorization': f'Bearer {openai_api_key}'
            }
            response = requests.post(url, json=payload, headers=headers)            
            if response.status_code == 200:
                return response
            else:
                response.raise_for_status()
        except Exception as e:
            if print_results:
                print(f'Attempt {attempt+1} failed: {e}')
            time.sleep(backoff_factor ** attempt)

In [1]:
text = "Once the text has been divided into manageable chunks, the next step is to create a summary for each chunk. This can be done by feeding each chunk into the LLM and asking it to generate a summary. The LLM will process each chunk and produce a summary that captures the main points and ideas of the chunk. This step ensures that the most important information from each chunk is retained. (You can set a limit for the LLM to not exceed a certain number of words for each summary)"

In [36]:
r = call_openai_tts()

In [37]:
audio_content = r.content
ipd.display(ipd.Audio(audio_content))

In [5]:
def call_openai_moderations(input,model='text-moderation-latest',max_retries=6, backoff_factor=1.5, openai_api_key=OPENAI_API_KEY, print_results=False):
    if not openai_api_key:
        raise ValueError("OpenAI API key is required.")

    for attempt in range(max_retries):
        try:
            url = 'https://api.openai.com/v1/moderations'
            payload = {
                'model': model,
                'input': input
            }
            headers = {
                'Content-Type': 'application/json',
                'Authorization': f'Bearer {openai_api_key}'
            }
            response = requests.post(url, json=payload, headers=headers)
            
            if response.status_code == 200:
                if print_results:
                    print(response.json())
                return response
            else:
                response.raise_for_status()
        except Exception as e:
            if print_results:
                print(f'Attempt {attempt+1} failed: {e}')
            time.sleep(backoff_factor ** attempt)

In [6]:
r = call_openai_moderations(text)

In [10]:
type(r)
r.json()['results']

[{'flagged': False,
  'categories': {'sexual': False,
   'hate': False,
   'harassment': False,
   'self-harm': False,
   'sexual/minors': False,
   'hate/threatening': False,
   'violence/graphic': False,
   'self-harm/intent': False,
   'self-harm/instructions': False,
   'harassment/threatening': False,
   'violence': False},
  'category_scores': {'sexual': 0.00011615458788583055,
   'hate': 1.5480063666473143e-05,
   'harassment': 0.00010946714610327035,
   'self-harm': 6.38433675703709e-06,
   'sexual/minors': 3.186627145623788e-05,
   'hate/threatening': 3.9824902842156007e-07,
   'violence/graphic': 0.00019347072520758957,
   'self-harm/intent': 2.597424463601783e-05,
   'self-harm/instructions': 5.7970473790192045e-06,
   'harassment/threatening': 6.9050788624736015e-06,
   'violence': 0.0007701614522375166}}]

In [10]:
def call_openai_requests(system_prompt, user_prompt, is_json=False, model_name='gpt-3.5-turbo-0125', temperature=0.7, max_tokens=3000, max_retries=6, backoff_factor=1.5, openai_api_key=OPENAI_API_KEY, print_results=False):
    for attempt in range(max_retries):
        try:
            #https://platform.openai.com/docs/api-reference/chat/create?lang=python
            messages = [{'role': 'system', 'content': system_prompt},
                        {'role': 'user', 'content': user_prompt}]
            url = 'https://api.openai.com/v1/chat/completions'
            if is_json:
                payload = json.dumps({
                    'model': model_name,
                    'messages': messages,
                    'temperature': temperature,
                    'max_tokens': max_tokens,
                    'response_format':{ "type": "json_object" },
                    # 'tool_choice': 
                    # # 'top_p':
                    
                    #### Other possible vars:#tools #top_p# frequency_penalty# #logit_bias#logprob s#top_logprobs  n# presence_penalty# response_format# stream# #stop# seed
            })
            else:
                payload = json.dumps({
                    'model': model_name,
                    'messages': messages,
                    'temperature': temperature,
                    'max_tokens': max_tokens,
            })

            headers = {'Content-Type': 'application/json',
                       'Authorization': f'Bearer {openai_api_key}'}
            response = requests.post(url, data=payload, headers=headers)
            if response.status_code == 200:
                if print_results:
                    print(response.json())
                return response
            else:
                raise Exception('Non-200 response received')
        except Exception as e:
            if print_results:
                print(f'Attempt {attempt+1} failed: {e}')
            time.sleep(backoff_factor ** attempt)

###################################################################################################
def call_mistralai_requests(system_prompt, user_prompt,is_json=False, model_name='mistral-medium-latest', temperature=0.7, max_tokens=3000, max_retries=6, backoff_factor=1.5, mistralai_api_key=MISTRALAI_API_KEY, print_results=False):
    for attempt in range(max_retries):
        try:
            #https://docs.mistral.ai/api/#operation/createChatCompletion
            messages = [{'role': 'system', 'content': system_prompt},
                        {'role': 'user', 'content': user_prompt}]
            url = 'https://api.mistral.ai/v1/chat/completions'
            if is_json:
                payload = json.dumps({
                    'model': model_name,
                    'messages': messages,
                    'temperature': temperature,
                    'max_tokens': max_tokens,
                    "safe_prompt": True,
                    'response_format':{ "type": "json_object" },                    
                #Other possible varriables:#top_p#stream#safe_prompt#random_seed
            })
            else:
                payload = json.dumps({
                'model': model_name,
                'messages': messages,
                'temperature': temperature,
                'max_tokens': max_tokens,
                "safe_prompt": True,
            })
            headers = {'Content-Type': 'application/json',
                       'Authorization': f'Bearer {mistralai_api_key}'}
            response = requests.post(url, data=payload, headers=headers)
            if response.status_code == 200:
                if print_results:
                    print(response.json())
                return response
            else:
                raise Exception('Non-200 response received')
        except Exception as e:
            if print_results:
                print(f'Attempt {attempt+1} failed: {e}')
            time.sleep(backoff_factor ** attempt)

#######################################################################
def call_ollama_requests(system_prompt, user_prompt, model_name='codellama:13b', temperature=0.7, max_tokens=2000, n=1, max_retries=6, backoff_factor=1.5, ollama_api_key=OLLAMA_API_KEY, print_results=False):
    for attempt in range(max_retries):
        try:
            url = 'https://sandbox-ollama.omnipeak.ai/api/chat'
            payload = json.dumps({
                'model': model_name,
                'messages': [{'role': 'system', 'content': system_prompt},
                             {'role': 'user', 'content': user_prompt}],
                'temperature': temperature,
                'max_tokens': max_tokens,
            })
            headers = {'Content-Type': 'application/json',
                       'Authorization': f'Basic {ollama_api_key}'}
            response = requests.post(url, data=payload, headers=headers)
            print(response)
            if response.status_code == 200:
                if print_results:
                    print(response.json())
                return response
            else:
                raise Exception('Non-200 response received')
        except Exception as e:
            if print_results:
                print(f'Attempt {attempt+1} failed: {e}')
            time.sleep(backoff_factor ** attempt)

###########################################
def call_azureopenai_requests(system_prompt, user_prompt, temperature=0.7, max_tokens=3000, max_retries=6, backoff_factor=1.5, azure_openai_api_key=AZURE_OPENAI_API_KEY, print_results=False):
    for attempt in range(max_retries):
        try:
            messages = [{'role': 'system', 'content': system_prompt},
                        {'role': 'user', 'content': user_prompt}]
            url = 'https://omnipeak-test.openai.azure.com/openai/deployments/test/chat/completions?api-version=2023-07-01-preview'
            payload = json.dumps({
                'messages': messages,
                'temperature': temperature,
                'max_tokens': max_tokens,
                #Other possible variables: #top_p #stream #safe_prompt #random_seed
            })
            headers = {'Content-Type': 'application/json',
                       'api-key': f'{azure_openai_api_key}'}
            response = requests.post(url, data=payload, headers=headers)
            if response.status_code == 200:
                if print_results:
                    print(response.json())
                return response
            else:
                raise Exception('Non-200 response received')
        except Exception as e:
            if print_results:
                print(f'Attempt {attempt+1} failed: {e}')
            time.sleep(backoff_factor ** attempt)

#######################################################################
def get_response_data_ollama(response):
    #This data format is active for OpenAI, AzureOPenAI , and MistralAI, it might need changes for Ollama
    response_json = response.json()
    content = response_json['message']['content']
    try:  #ollama sometimes do not return prompt tokens
        prompt_tokens = response_json['usage']['prompt_eval_count']
    except:
        prompt_tokens = 0
    try:
        completion_tokens = response_json['usage']['eval_count']
    except:
        completion_tokens = 0
    return content, prompt_tokens, completion_tokens

#######################################################################
def get_response_data(response):
    #This data format is active for OpenAI, AzureOPenAI , and MistralAI, it might need changes for Ollama
    response_json = response.json()
    content = response_json['choices'][0]['message']['content']
    # prompt_tokens =
    prompt_tokens = response_json['usage']['prompt_tokens']
    completion_tokens = response_json['usage']['completion_tokens']
    return content, prompt_tokens, completion_tokens

#######################################################################
def call_llm(llm_provider, model_name, system_prompt, user_prompt, temperature=0.7, max_tokens=3000, max_retries=6, backoff_factor=1.5, print_results=False):
    """a general wat to call all the LLM providers with one function,
    watch that ollama responses are slightly different, but this is considered
    in the function get_response_data_ollama
    
    returns (response, prompt_tokens, compeletion_tokens)"""
    if llm_provider == 'openai':
        response = call_openai_requests(system_prompt,user_prompt,model_name=model_name, temperature=temperature,max_tokens=max_tokens, max_retries=max_retries, backoff_factor=backoff_factor,openai_api_key=OPENAI_API_KEY, print_results=print_results) 
        data = get_response_data(response)
        
    elif llm_provider == 'azureopenai':
        response=call_azureopenai_requests(system_prompt,user_prompt,temperature=temperature,max_tokens=max_tokens, max_retries=max_retries, backoff_factor=backoff_factor,azure_openai_api_key=AZURE_OPENAI_API_KEY, print_results=print_results)#no model name in Azure OPenAI 
        data =get_response_data(response)

    elif llm_provider == 'mistralai':
        response=call_mistralai_requests(system_prompt,user_prompt,model_name=model_name, temperature=temperature,max_tokens=max_tokens, max_retries=max_retries, backoff_factor=backoff_factor,mistralai_api_key=MISTRALAI_API_KEY, print_results=print_results)
        data=get_response_data(response)

    elif llm_provider == 'ollama':
        response=call_ollama_requests(system_prompt,user_prompt,model_name=model_name, temperature=temperature,max_tokens=max_tokens, max_retries=max_retries, backoff_factor=backoff_factor,ollama_api_key=OLLAMA_API_KEY, print_results=print_results)
        data=get_response_data_ollama(response) 

    return data

In [18]:
system_prompt = 'Act as a copywriter for an AI company'
user_prompt = 'Write an email to convince a client to buy AI products. Give response in JSON'
http_response = call_mistralai_requests(system_prompt, user_prompt,model_name="mistral-tiny-2312")

In [19]:
print(http_response.json())

{'id': 'd441c98125674004809d46c8e768b291', 'object': 'chat.completion', 'created': 1714121919, 'model': 'mistral-tiny-2312', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '{\n "subject": "Transform Your Business with Our Cutting-Edge AI Solutions",\n "body": "Dear Valued Client,\\n\\nWe hope this email finds you well. At [AI Company Name], we are constantly innovating and pushing the boundaries of what\'s possible with Artificial Intelligence (AI). We believe that AI is no longer a nice-to-have but a must-have for businesses looking to stay competitive in today\'s fast-paced world.\\n\\nOur AI products are designed to streamline your business processes, increase efficiency, and drive growth. Here are just a few ways we can help:\\n\\n1. Predictive Analytics: Our AI algorithms can analyze your data and provide insights into customer behavior, market trends, and operational efficiency, enabling you to make data-driven decisions and improve your bottom line.\\n\\n2.

In [18]:
system_prompt = 'Act as a copywriter for an AI company'
user_prompt = 'Write an email to convince a client to buy AI products. Give response in JSON'

http_response = call_openai_requests(system_prompt, user_prompt)
response = get_response_data(response)


In [29]:
print(http_response.json()['choices'][0]['message']['content'])

{
  "subject": "Unlock the Power of AI for Your Business",
  "recipient": "Client",
  "body": "Dear Client, \n\nI hope this email finds you well. I wanted to reach out to introduce you to our cutting-edge AI products that can revolutionize the way your business operates. By leveraging the power of artificial intelligence, you can streamline processes, enhance decision-making, and drive innovation like never before. \n\nOur AI solutions are designed to meet the specific needs of your industry and can help you stay ahead of the competition. From predictive analytics to personalized customer experiences, our products are tailored to deliver tangible results for your business. \n\nI would love to schedule a demo to show you the incredible capabilities of our AI technology and discuss how it can be customized to fit your unique business needs. Don't miss out on this opportunity to transform your operations and drive growth with AI. \n\nLet's take your business to the next level together. Co

In [ ]:
import time
import requests
import json

# Constants
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
MISTRALAI_API_KEY = os.getenv('MISTRALAI_API_KEY')
OLLAMA_API_KEY = os.getenv('OLLAMA_API_KEY')
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')

# API URLs
API_URLS = {
    'openai': 'https://api.openai.com/v1/chat/completions',
    'mistralai': 'https://api.mistral.ai/v1/chat/completions',
    'ollama': 'https://sandbox-ollama.omnipeak.ai/api/chat',
    'azureopenai': 'https://omnipeak-test.openai.azure.com/openai/deployments/test/chat/completions?api-version=2023-07-01-preview'
}

def make_request(api_key, url, payload, headers, max_retries=6, backoff_factor=1.5, print_results=False):
    for attempt in range(max_retries):
        try:
            response = requests.post(url, data=payload, headers=headers)
            if response.status_code == 200:
                if print_results:
                    print(response.json())
                return response
            else:
                raise Exception('Non-200 response received')
        except Exception as e:
            if print_results:
                print(f'Attempt {attempt+1} failed: {e}')
            time.sleep(backoff_factor ** attempt)

def call_api(api_name, system_prompt, user_prompt, model_name, temperature=0.7, max_tokens=3000, print_results=False):
    messages = [{'role': 'system', 'content': system_prompt}, 
                {'role': 'user', 'content': user_prompt}
                ]
    payload = json.dumps({
        'model': model_name,
        'messages': messages,
        'temperature': temperature,
        'max_tokens': max_tokens
    })
    headers = {'Content-Type': 'application/json', 'Authorization': f'Bearer {api_key}'}
    url = API_URLS[api_name]
    api_key = globals()[f'{api_name.upper()}_API_KEY']
    return make_request(api_key, url, payload, headers, print_results=print_results)

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
print(run_conversation())

In [ ]:
##############################################################
def send_email(email_content,email_subject,email_receiver,email_sender=AGENT_EMAIL, email_password=AGENT_EMAIL_PASSWORD, print=False):
    context = ssl.create_default_context()  # Create a SSL context to ensure a secure connection
    try:
        # Connect securely to the Gmail SMTP server and send the email
        with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
            smtp.login(email_sender, email_password)  # Log in to the SMTP server
            em = EmailMessage()
            em['From'] = email_sender
            em['To'] = email_receiver
            em['Subject'] = email_subject
            em.set_content(email_content)  # Set the email content
            smtp.sendmail(email_sender, email_receiver, em.as_string())  # Send the email
            if print:
                print("Email sent successfully!")
    except Exception as e:
        if print:
            print(f"Failed to send email: {e}")  # Print the exception if sending failed and print is True

#################################################################
def send_email_with_html(email_content, email_subject, email_reciever, email_sender=AGENT_EMAIL, email_password=AGENT_EMAIL_PASSWORD, print=False):
    """
    Sends an HTML email using SMTP, specifically configured for Gmail's SMTP server.

    Parameters:
    - email_content (str): The HTML content of the email to send.
    - email_subject (str): The subject line of the email.
    - email_reciever (str): The email address of the recipient.
    - email_sender (str): The email address of the sender. Defaults to AGENT_EMAIL.
    - email_password (str): The password for the sender's email account. Defaults to AGENT_EMAIL_PASSWORD.
    - print (bool): If True, prints a message indicating the success or failure of the email sending operation.

    This function constructs an HTML email message from the given parameters and sends it using Gmail's SMTP server.
    It first establishes a connection to the server with STARTTLS for security, logs in with the sender's credentials,
    and then sends the HTML email. If the 'print' parameter is True, it will output a message upon success or failure.
    """
    try:
        # Connect to the Gmail SMTP server with STARTTLS for security
        with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
            smtp.ehlo()  # Identify ourselves to the SMTP server
            smtp.starttls()  # Encrypt the email content
            smtp.login(email_sender, email_password)  # Log in to the server

            message = MIMEMultipart()  # Create a MIMEMultipart message
            message['From'] = email_sender
            message['To'] = email_reciever
            message['Subject'] = email_subject

            html_part = MIMEText(email_content, 'html')  # Create the HTML part of the email
            message.attach(html_part)  # Attach the HTML part

            smtp.sendmail(email_sender, email_reciever, message.as_string())  # Send the email
            if print:
                print("Email sent successfully!")
    except Exception as e:
        if print:
            print(f"Failed to send email: {e}")  # Print the exception if sending failed and print is True